# Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, html, dcc, dash_table
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output

# Load The Data

In [2]:
data = pd.read_csv('sales data.csv')

In [3]:
data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


# Create the Layout of the app

In [4]:
branches = data.Branch.unique()
branches.sort()
filter_options = []
options=[]
for i in branches:
    option = {'label': i, 'value': i}
    filter_options.append(option)
    options.append(i)

In [5]:
app = JupyterDash()
app.layout = html.Div([
    html.Div([
            html.Label('Select Branch'),
            dcc.Dropdown(filter_options, 'A', clearable = False , id = 'input')],
            style = {'display': 'inline-block', 'width': '200px', 'margin': '10px', 'vertical-align': 'top'}),
    
    html.Div([
        dcc.Graph(style = {'display': 'inline-block', 'width': '680px', 'height': '500px', 'margin': '20px'},
                  id = 'product_line_bp_1'),
        dcc.Graph(style = {'display': 'inline-block', 'width': '680px', 'height': '500px', 'margin': '20px 20px 20px 80px'},
                  id = 'gender_bp_1')
    ]),
    
    html.Div([
        dcc.Graph(style = {'display': 'inline-block', 'width': '1460px', 'margin': '20px'}, id = 'time_lp')
    ]),
    
    html.Div([
        dcc.Graph(style = {'display': 'inline-block', 'width': '680px', 'height': '500px', 'margin': '20px'},
                  id = 'product_line_bp_2'),
        dcc.Graph(style = {'display': 'inline-block', 'width': '680px', 'height': '500px', 'margin': '20px 20px 20px 80px'},
                  id = 'gender_bp_2')
    ]),
])

D:\anaconda\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



# Graph showing 'COGS in each category'

In [6]:
@app.callback(
        Output(component_id = 'product_line_bp_1', component_property = 'figure'),
        Input(component_id = 'input', component_property = 'value')
    )
def product_line_bp_1(branch_input = 'A'):
    bp_1_data = data[data.Branch == branch_input].groupby('Product line').sum('cogs').reset_index()
    bp_1 = px.bar(bp_1_data, x = 'Product line', y = 'cogs', color = 'Product line', labels = {'cogs': 'Cost of goods sold'},
                 title = 'COGS in each category')
    bp_1.update_xaxes(linecolor = 'black')
    bp_1.update_yaxes(linecolor = 'black')
    bp_1.update_layout(plot_bgcolor = '#FFFFFF')
    return bp_1

# Graph showing 'COGS to each gender'

In [7]:
@app.callback(
        Output(component_id = 'gender_bp_1', component_property = 'figure'),
        Input(component_id = 'input', component_property = 'value')
    )
def gender_bp_1(branch_input):
    if len(branch_input) == 0:
        branch_input = 'A'
    bp_2_data = data[data.Branch == branch_input].groupby('Gender').sum('cogs').reset_index()
    bp_2 = px.bar(bp_2_data, x = 'Gender', y = 'cogs', color = 'Gender', labels = {'cogs': 'Cost of goods sold'},
                  title = 'COGS to each Gender')
    bp_2.update_xaxes(linecolor = 'black')
    bp_2.update_yaxes(linecolor = 'black')
    bp_2.update_layout(plot_bgcolor = '#FFFFFF')
    return bp_2

# Graph showing 'Number of customers in the store based on time of day'

In [8]:
@app.callback(
        Output(component_id = 'time_lp', component_property = 'figure'),
        Input(component_id = 'input', component_property = 'value')
    )
def time_lp(branch_input = 'A'):
    lp_1_data = pd.DataFrame(data[data.Branch == branch_input].Time.value_counts())
    lp_1_data.reset_index(inplace = True)
    lp_1_data.columns = ['Time', 'Number of Customers']
    lp_1_data.Time = lp_1_data.Time.astype('datetime64[ns]')
    lp_1_data = lp_1_data.resample('H', on = 'Time')['Number of Customers'].sum()
    lp_1_data = pd.DataFrame(lp_1_data)
    lp_1_data.reset_index(inplace = True)
    lp_1_data.Time = lp_1_data.Time.dt.hour
    lp_1 = px.line(lp_1_data, x = 'Time', y = 'Number of Customers',
                   labels = {'Time': 'Hour of day'},
                   title = 'Number of Customers in the store based on time of day')
    lp_1.update_xaxes(range = [9.5, 20.5], linecolor = 'Black', ticks = 'inside', tickcolor = 'black')
    lp_1.update_yaxes(linecolor = 'black', ticks = 'inside', tickcolor = 'black')
    lp_1.update_layout(plot_bgcolor = '#FFFFFF')
    return lp_1


# Graph Showing 'Number of items sold in each category'

In [9]:
@app.callback(
        Output(component_id = 'product_line_bp_2', component_property = 'figure'),
        Input(component_id = 'input', component_property = 'value')
    )
def product_line_bp(branch_input = 'A'):
    bp_3_data = data[data.Branch == branch_input].groupby('Product line').sum('Quantity').reset_index()
    bp_3 = px.bar(bp_3_data, x = 'Product line', y = 'Quantity', color = 'Product line',
                 labels = {'Quantity': 'Number of items sold'}, title = 'Number of items sold in each category')
    bp_3.update_xaxes(linecolor = 'black')
    bp_3.update_yaxes(linecolor = 'black')
    bp_3.update_layout(plot_bgcolor = '#FFFFFF')
    return bp_3

# Graph showing 'Number of customers based on gender'

In [10]:
@app.callback(
        Output(component_id = 'gender_bp_2', component_property = 'figure'),
        Input(component_id = 'input', component_property = 'value')
    )
def gender_bp_2(branch_input = 'A'):
    bp_4_data = pd.DataFrame(data[data.Branch == branch_input].Gender.value_counts())
    bp_4_data.reset_index(inplace = True)
    bp_4_data.columns = ['Gender', 'Number of customers']
    bp_4 = px.bar(bp_4_data, x = 'Gender', y = 'Number of customers', color = 'Gender', labels = {'cogs': 'Cost of goods sold'},
                  title = 'Number of customers based on gender')
    bp_4.update_xaxes(linecolor = 'black')
    bp_4.update_yaxes(linecolor = 'black')
    bp_4.update_layout(plot_bgcolor = '#FFFFFF')
    return bp_4

# Launch the app

In [11]:
app.run_server()

Dash app running on http://127.0.0.1:8050/


C:\Users\dell\AppData\Local\Temp\ipykernel_15064\3433510272.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\dell\AppData\Local\Temp\ipykernel_15064\3433510272.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\dell\AppData\Local\Temp\ipykernel_15064\3433510272.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\dell\AppData\Local\Temp\ipykernel_15064\3433510272.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a f